# Dependencies

In [2]:
%%capture
!pip install -U sentence-transformers
!pip install emoji
!pip install apify-client
!pip install -U deep-translator

In [3]:
# Standard library imports
import os
import time
import re
import random

# Third-party library imports
import pandas as pd
import numpy as np
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import emoji

# Apify and Google-specific imports
from apify_client import ApifyClient
from google.colab import drive, userdata
import googleapiclient.discovery
import googleapiclient.errors

In [4]:
# Connecting Drive
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/ADSP/data/'

Mounted at /content/drive


# Comments

## Digimind

In [ ]:
# Load Digimind Data
social_media_df = pd.read_excel(data_folder + 'IT-Lavazza&Tiny_Digimind_SocialListening_2022-2024.xlsx')
social_media_df.head(3)

,id,Title,Detail,Link,Source,Update date,Publish date,Sentiment,Ranking,Media type,...,Reach,Interactions,Notes,Author name,Author handle (@username),Author URL,Gender,Age,Bio,City
0,1875205,Simplicity full of colour. Lavazza A Modo Mio ...,Simplicity full of colour. Lavazza A Modo Mio ...,https://www.facebook.com/120824298064569_61677...,facebook.com,17/10/2024 10:18:22,24/05/2023 10:00:10,Positive,6,Facebook,...,613,3.0,NaN,Lavazza A Modo Mio Malta,amodomiomalta,https://www.facebook.com/120824298064569,NaN,NaN,NaN,NaN
1,1885275,Macchine per il caffè espresso: sono queste le...,Per chi preferisce sorseggiare un espresso anc...,https://www.greenme.it/lifestyle/sai-cosa-comp...,greenme.it,17/10/2024 10:18:22,25/05/2023 15:35:41,Positive,8,Blogs,...,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1965935,Macchina da caffè Lavazza A Modo Mio Tiny Eco ...,Macchina da caffè Lavazza A Modo Mio Tiny Eco ...,https://www.facebook.com/164744940227074_63065...,facebook.com,17/10/2024 10:18:24,29/05/2023 10:09:50,No sentiment,4,Facebook,...,75,0.0,NaN,Risparmio Felice,risparmiofelice,https://www.facebook.com/164744940227074,NaN,NaN,NaN,NaN


In [ ]:
# NaN values
social_media_df.isna().sum(), social_media_df.shape

(id                             0
 Title                          0
 Detail                         0
 Link                           0
 Source                         0
 Update date                    0
 Publish date                   0
 Sentiment                      0
 Ranking                        0
 Media type                     0
 Tags                           0
 Country                      177
 Language                       5
 Audience                       0
 Reach                          0
 Interactions                 157
 Notes                        577
 Author name                  232
 Author handle (@username)    232
 Author URL                   157
 Gender                       426
 Age                          577
 Bio                          474
 City                         549
 dtype: int64,
 (577, 24))

In [ ]:
# Columns
social_media_df = social_media_df[['Detail', 'Link', 'Publish date', 'Sentiment', 'Media type', 'Source', 'Language']].copy()

social_media_df.rename(columns={
    'Detail': 'postCaption',
    'Link': 'postUrl',
    'Publish date': 'postTimeStamp'
}, inplace=True)
social_media_df.head()

,postCaption,postUrl,postTimeStamp,Sentiment,Media type,Source,Language
0,Simplicity full of colour. Lavazza A Modo Mio ...,https://www.facebook.com/120824298064569_61677...,24/05/2023 10:00:10,Positive,Facebook,facebook.com,English
1,Per chi preferisce sorseggiare un espresso anc...,https://www.greenme.it/lifestyle/sai-cosa-comp...,25/05/2023 15:35:41,Positive,Blogs,greenme.it,Italian
2,Macchina da caffè Lavazza A Modo Mio Tiny Eco ...,https://www.facebook.com/164744940227074_63065...,29/05/2023 10:09:50,No sentiment,Facebook,facebook.com,Italian
3,Ti informiamo che Monclick con sede in Via Mar...,https://www.ebay.it/itm/144716379715?campid=53...,29/05/2023 10:09:50,No sentiment,News,ebay.it,Italian
4,"Lavazza A Modo Mio Tiny Eco. Beautiful, compac...",https://www.facebook.com/120824298064569_61959...,29/05/2023 10:00:09,Positive,Facebook,facebook.com,English


In [ ]:
# Duplicate columns postCaption, postTimestamp since we don't have
# text(commene/review), commentTimestamp
social_media_df['text'] = social_media_df['postCaption']
social_media_df['commentTimeStamp'] = social_media_df['postTimeStamp']

In [ ]:
# Media type
social_media_df.groupby('Media type').count().sort_values(by='postUrl', ascending=False)['postUrl']

,postUrl
Media type,
Facebook,214
News,136
X (Twitter),111
Instagram,88
Video,13
Blogs,8
TikTok,7


In [ ]:
# Video source
social_media_df[social_media_df['Media type'] == 'Video'].Source.unique()

array(['youtube.com'], dtype=object)

In [ ]:
# Video -> YouTube for consistency
social_media_df.loc[social_media_df['Media type'] == 'Video', 'Media type'] = 'YouTube'

In [ ]:
# Add coffe machine model
social_media_df['model'] = 'tinyeco'

In [ ]:
# Languages
languages = list(social_media_df['Language'].unique())

print(f'Languages: {languages}')

for lang in languages:
  try:
    caption = list(social_media_df[social_media_df["Language"] == lang]["text"])[0]
    print('\n\n')
    print(f'{lang}:\n{caption}')
  except:
    continue

Languages: ['English', 'Italian', 'German', 'French', 'Hungarian', 'Albanian', nan, 'Bulgarian', 'Hebrew', 'Portuguese', 'Croatian', 'Romanian', 'Greek']



English:
Simplicity full of colour. Lavazza A Modo Mio Tiny Eco is the best way to start your day!                                        

Shop all the Lavazza range online at: www.thestore.mt⁠

#LavazzaAModoMio #LavazzaBaristaTechnology
.
.
.
*TUV Austria logo is referred to the capsules. COMPOSTABLE CAPSULES certified EN 13432:2002 for industrial composting by TUV Austria. 
Simplicity full of colour. Lavazza A Modo Mio Tiny Eco is the best way to start your day!                                        

Shop all the Lavazza range online at: www.thestore.mt⁠

#LavazzaAModoMio #LavazzaBaristaTechnology
.
.
.
*TUV Austria logo is referred to the capsules. COMPOSTABLE CAPSULES certified EN 13432:2002 for industrial composting by TUV Austria.



Italian:
Per chi preferisce sorseggiare un espresso anche a casa, esistono ormai sul merca

In [ ]:
# Some of translations are wrong due to the presence of emoji (we are going retranslate the text and the postCaption)
social_media_df.drop(columns=['Language'], inplace=True)

In [ ]:
# Drop Sentiment since it refers to an ADV
social_media_df.drop(columns=['Sentiment'], inplace=True)

In [ ]:
# These are not real comments
social_media_df['isComment'] = False

In [ ]:
social_media_df.to_csv(data_folder + 'digimind.csv', sep='\t', index=False)

## Instagram

In [ ]:
# Function to determine the product model based on hashtags
def define_model(hashtags):
  if hashtags == pd.NA:
    return pd.NA

  hashtags = [hashtag.lower() for hashtag in hashtags]
  if 'lavazzatinyeco' in hashtags:
    return 'tinyeco'
  elif 'lavazzadesea' in hashtags:
    return 'desea'
  elif 'amodomiosmeg' in hashtags:
    return 'amodomiosmeg'
  return pd.NA

# Function to fetch Instagram posts using the Apify Client
def fetch_instagram_posts(client, hashtags, resultsLimit, extra_posts_df=None):
  # Define the input parameters for the Apify actor
  run_input = {
    "hashtags": hashtags,
    "resultsLimit": resultsLimit,
  }

  # Call the Apify actor to fetch posts
  try:
    run = client.actor('reGe1ST3OBgYZSsZJ').call(run_input=run_input)
  except Exception as e:
    raise RuntimeError(f"Error fetching Instagram posts: {e}")

  # Extract post information
  posts = []
  for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    posts.append({
        'postCaption': item.get('caption', pd.NA),
        'commentsCount': pd.NA if item.get('commentsCount', pd.NA) == 0 else item.get('commentsCount', pd.NA),
        'postTimeStamp': item.get('timestamp', pd.NA),
        'postUrl': item.get('url', pd.NA),
        'model': define_model(item.get('hashtags', pd.NA))
    })

  # Create a DataFrame of posts and remove duplicates
  post_df = pd.DataFrame(posts).dropna(subset=['postUrl']).reset_index(drop=True)

  if extra_posts_df is not None:
    post_df = pd.concat([post_df, extra_posts_df], ignore_index=True).drop_duplicates(subset=['postUrl'])

  return post_df


# Function to fetch comments for Instagram posts
def fetch_instagram_comments(client, post_urls, resultsLimit):

    run_input = {
        "directUrls": post_urls,
        "resultsLimit": resultsLimit
    }

    try:
      run = client.actor('SbK00X0JYCPblD2wp').call(run_input=run_input)
    except Exception as e:
      raise RuntimeError(f"Error fetching Instagram comments: {e}")

    comments = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
      comments.append({
          'text': item.get('text', pd.NA),
          'postUrl': item.get('postUrl', pd.NA),
          'commentTimeStamp': item.get('timestamp', pd.NA)
      })
    return pd.DataFrame(comments).dropna(subset=['text']).reset_index(drop=True)


# Main function
def get_instagram_comments(hashtags, resultsLimit=200, extra_posts_df=None):

    try:
      # Load API token
      APIFY_TOKEN = userdata.get('APIFY_TOKEN')
      client = ApifyClient(APIFY_TOKEN)

      # File paths
      insta_posts_file = os.path.join(data_folder, 'insta_posts.csv')
      insta_comments_file = os.path.join(data_folder, 'insta_comments.csv')
      insta_comments_per_post_file = os.path.join(data_folder, 'insta_comments_per_post.csv')

      # Fetch or load posts
      if os.path.exists(insta_posts_file):
        print("Posts already taken")
        insta_posts_df = pd.read_csv(insta_posts_file, sep='\t')
      else:
        insta_posts_df = fetch_instagram_posts(client, hashtags, resultsLimit, extra_posts_df)
        insta_posts_df.to_csv(insta_posts_file, sep='\t', index=False)

      # Fetch or load comments
      if os.path.exists(insta_comments_file):
        print("Comments already taken")
        insta_comments_df = pd.read_csv(insta_comments_file, sep='\t')
      else:
        post_urls = list(insta_posts_df.postUrl)
        insta_comments_df = fetch_instagram_comments(client, post_urls, 20)
        insta_comments_df.to_csv(insta_comments_file, sep='\t', index=False)

      # Aggregate posts and comments
      if os.path.exists(insta_comments_per_post_file):
        print("Aggregated data already available")
        insta_comments_per_post_df = pd.read_csv(insta_comments_per_post_file, sep='\t')
      else:
        insta_comments_per_post_df = pd.merge(
            insta_comments_df, insta_posts_df, on="postUrl", how="left"
        )
        insta_comments_per_post_df.to_csv(insta_comments_per_post_file, sep='\t', index=False)

      return insta_comments_per_post_df

    # Exception handling
    except FileNotFoundError as fnf_error:
        print(fnf_error)
    except KeyError as key_error:
        print(key_error)
    except RuntimeError as runtime_error:
        print(runtime_error)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
# Get posts link from external sources (Digimind)
extra_posts_df = pd.read_csv(data_folder + 'digimind.csv', sep='\t')
extra_posts_df = extra_posts_df[extra_posts_df['Media type'] == 'Instagram'][['postCaption', 'postTimeStamp', 'postUrl', 'model']]
extra_posts_df['commentsCount'] = np.NaN

extra_posts_df.head()

,postCaption,postTimeStamp,postUrl,model,commentsCount
5,Discover a mesmerizing fusion of style and fun...,31/05/2023 13:54:37,https://www.instagram.com/p/Cs6BW3rtBOu,tinyeco,NaN
7,Lavazza a modo mio tiny eco #macchinadelcaff...,31/05/2023 13:51:50,https://www.instagram.com/p/Cs6BCYGLHx1,tinyeco,NaN
11,Experience the rich and creamy taste of the D...,05/06/2023 16:03:58,https://www.instagram.com/p/CtHIIpUILrk,tinyeco,NaN
13,Nuova lavazza Tiny \n#macchinecaffè #lavazza #...,09/06/2023 09:00:36,https://www.instagram.com/p/CtQq3Rpty0U,tinyeco,NaN
16,Lavazza’s tiny companion. \nSmall in size but ...,14/06/2023 14:06:28,https://www.instagram.com/p/CteF14foKFM,tinyeco,NaN


In [ ]:
# Hashtags
hashtags = ['lavazzatinyeco', 'lavazzadesea', 'amodomiosmeg']

insta_comments_per_post_df = get_instagram_comments(hashtags, resultsLimit=200, extra_posts_df=extra_posts_df)
insta_comments_per_post_df.head()

Posts already taken
Comments already taken
Aggregated data already available


,text,postUrl,commentTimestamp,postCaption,commentsCount,postTimestamp,model
0,❤️,https://www.instagram.com/p/Ct1EteaIVUd,2023-08-27T07:35:30.000Z,A tiny machine for mighty practicality! \n\nOr...,NaN,23/06/2023 12:19:07,tinyeco
1,Adoro il colore,https://www.instagram.com/reel/CxtPqQLr5D9,2023-10-01T19:03:35.000Z,Macchina del caffè \n\nEcco finalmente la nost...,NaN,27/09/2023 21:45:34,tinyeco
2,😍🙌,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-29T09:58:03.000Z,Macchina del caffè \n\nEcco finalmente la nost...,NaN,27/09/2023 21:45:34,tinyeco
3,Mi piace il colore . Diverso dal solito,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-28T14:11:20.000Z,Macchina del caffè \n\nEcco finalmente la nost...,NaN,27/09/2023 21:45:34,tinyeco
4,Colore 🔝,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-28T12:09:52.000Z,Macchina del caffè \n\nEcco finalmente la nost...,NaN,27/09/2023 21:45:34,tinyeco


In [ ]:
insta_comments_per_post_df.isna().sum(), insta_comments_per_post_df.shape

(text                 0
 postUrl              0
 commentTimestamp     0
 postCaption          0
 commentsCount       47
 postTimestamp        0
 model                0
 dtype: int64,
 (84, 7))

In [ ]:
# Drop commentsCount
insta_comments_per_post_df.drop(columns=['commentsCount'], inplace=True)

In [ ]:
# Set Media type to Instagram
insta_comments_per_post_df['Media type'] = 'Instagram'

In [ ]:
# Set isComment Flag to True
insta_comments_per_post_df['isComment'] = True

In [ ]:
# Ser Source to instagram.com
insta_comments_per_post_df['Source'] = 'instagram.com'

In [ ]:
insta_comments_per_post_df.to_csv(data_folder + 'instagram.csv', sep='\t', index=False)

## YouTube

In [ ]:
# Chose the videos
tiny_eco_videos = ['https://www.youtube.com/watch?v=ga2vKr0Af7k&t=26s&ab_channel=TRONY',
                   'https://www.youtube.com/watch?v=Jdw0B7HyPqI&ab_channel=HowToKillAnHour']

jolie_videos = ['https://www.youtube.com/watch?v=wP6l4MD1tTc&ab_channel=StefanoSogus',
               'https://www.youtube.com/watch?v=-rnRFbdJ1ZM&ab_channel=ItalianCoffeeVentures']

jolie_plus_videos = ['https://www.youtube.com/watch?v=Yj9lez3H5xg&ab_channel=ZoDaDesign']

jolie_and_milk_videos = ['https://www.youtube.com/watch?v=j53U692w3ik&ab_channel=TecHsia']

idola_videos = ['https://www.youtube.com/watch?v=wRBi_dANw5o&ab_channel=Bapervids']

desea_video = ['https://www.youtube.com/watch?v=ztoBH-cNI5E&ab_channel=Esploratore',
               'https://www.youtube.com/watch?v=JDC9-jq-k1k&ab_channel=YoussefFouad',
               'https://www.youtube.com/watch?v=HWFIKgQH6Ls&ab_channel=Stefanuzzo1992Rossano']

voicy_videos = ['https://www.youtube.com/watch?v=DHyWKYoX-xE&ab_channel=TechDale',
                'https://www.youtube.com/watch?v=Bq0Do-6F8QU&ab_channel=OttoCliman',
                'https://www.youtube.com/watch?v=Ox2c0GOYA8g&ab_channel=Poderak',
                'https://www.youtube.com/watch?v=CoifSvImZxs&ab_channel=99Caff%C3%A8L%27Aquila',
                'https://www.youtube.com/watch?v=BM2G2qaBP5g&ab_channel=AlessandroBaldissera',
                'https://www.youtube.com/watch?v=ecUd44qSPk4&ab_channel=Techprincess']

youtube_videos = {}
youtube_videos['tinyeco'] = tiny_eco_videos
youtube_videos['jolie'] = jolie_videos
youtube_videos['jolieplus'] = jolie_plus_videos
youtube_videos['jolie&milk'] = jolie_and_milk_videos
youtube_videos['idola'] = idola_videos
youtube_videos['desea'] = desea_video
youtube_videos['voicy'] = voicy_videos

In [ ]:
def get_youtube_comments(youtube_videos=youtube_videos, resultsLimit = 500, extra_video=None):

  # Configure YouTube request
  api_service_name = "youtube"
  api_version = "v3"
  YOUTUBE_TOKEN = userdata.get('YOUTUBE_TOKEN')

  youtube = googleapiclient.discovery.build(
      api_service_name, api_version, developerKey=YOUTUBE_TOKEN)

  # Define Dataframe
  youtube_comments_df = pd.DataFrame(columns=['commentTimeStamp', 'text','videoUrl','model', 'Media type'])

  # Add extra video
  if extra_video:
    for model in extra_video.keys():
      if model in youtube_videos.keys():
        youtube_videos[model].extend(extra_video[model])

  for model, videos in youtube_videos.items():

    # Creazione del DataFrame con estrazione diretta dell'ID
    youtube_url_id_df = pd.DataFrame(
        [(url, url.split('=')[1].split('&')[0]) for url in videos],
        columns=['videoUrl', 'videoId']
    ).drop_duplicates(subset='videoId').reset_index(drop=True)

    # Lista pulita di tuple (se necessaria)
    youtube_url_id = list(youtube_url_id_df.itertuples(index=False, name=None))

    for (video_url, video_id) in youtube_url_id:

      # Execute the request
      request = youtube.commentThreads().list(
          part="snippet",
          videoId=video_id,
          maxResults=resultsLimit
      )
      try:
        response = request.execute()
      except googleapiclient.errors.HttpError as e:
        continue

      # Elaborate the response
      for item in response['items']:

        new_row = {
          'commentTimeStamp': item.get('snippet', {}).get('topLevelComment', {}).get('snippet', {}).get('updatedAt', pd.NA),
          'text': item.get('snippet', {}).get('topLevelComment', {}).get('snippet', {}).get('textOriginal', pd.NA),
          'videoUrl': video_url,
          'model': model,
          'Media type': 'YouTube',
        }

        # Append the new row to the DataFrame
        youtube_comments_df = pd.concat([youtube_comments_df, pd.DataFrame([new_row])], ignore_index=True)

  return youtube_comments_df

In [ ]:
extra_video_df = pd.read_csv(data_folder + 'digimind.csv', sep='\t')
extra_video_df = extra_video_df[extra_video_df['Media type'] == 'YouTube']

extra_video = {}
for model in extra_video_df['model'].unique():
    extra_video[model] = list(extra_video_df[extra_video_df['model'] == model]['postUrl'])
extra_video

{'tinyeco': ['http://www.youtube.com/watch?v=5eyCzPFsM_w',
  'http://www.youtube.com/watch?v=Jdw0B7HyPqI',
  'http://www.youtube.com/watch?v=bjuig1TFHCw',
  'http://www.youtube.com/watch?v=zrMu31LfQZ0',
  'http://www.youtube.com/watch?v=_RaoKueShKQ',
  'http://www.youtube.com/watch?v=CCVDMDYIARQ',
  'http://www.youtube.com/watch?v=Gk5vqpEDuBw',
  'http://www.youtube.com/watch?v=l4-6rHyjQ1w',
  'http://www.youtube.com/watch?v=ga2vKr0Af7k',
  'http://www.youtube.com/watch?v=qBxIjxIFfM4',
  'http://www.youtube.com/watch?v=8MAOFOTTf-U',
  'http://www.youtube.com/watch?v=sd8hLh2lj3A',
  'http://www.youtube.com/watch?v=JYZEvOwj_Po']}

In [ ]:
youtube_comments_df = get_youtube_comments(youtube_videos=youtube_videos,
                                           resultsLimit = 500,
                                           extra_video=extra_video)
youtube_comments_df.head()

,commentTimeStamp,text,videoUrl,model,Media type
0,2024-11-02T08:44:01Z,"Pratica e caffè buono, mi dispiace che non fan...",https://www.youtube.com/watch?v=ga2vKr0Af7k&t=...,tinyeco,YouTube
1,2024-06-24T18:04:38Z,la macchina andrebbe anche bene ma questi lumi...,https://www.youtube.com/watch?v=ga2vKr0Af7k&t=...,tinyeco,YouTube
2,2024-05-28T17:09:24Z,vebbè ma l'acqua di scarico dove va a finire ????,https://www.youtube.com/watch?v=ga2vKr0Af7k&t=...,tinyeco,YouTube
3,2024-05-18T21:59:00Z,"È una schifezza, ha un serbatoio piccolissimo,...",https://www.youtube.com/watch?v=ga2vKr0Af7k&t=...,tinyeco,YouTube
4,2024-01-24T10:30:37Z,Davvero una ottima macchina da caffè,https://www.youtube.com/watch?v=ga2vKr0Af7k&t=...,tinyeco,YouTube


In [ ]:
youtube_comments_df.isna().sum(), youtube_comments_df.shape

(commentTimeStamp    0
 text                0
 videoUrl            0
 model               0
 Media type          0
 dtype: int64,
 (493, 5))

In [ ]:
# Comments X Model
youtube_comments_df.groupby('model').count().sort_values(by='text', ascending=False)['text']

,text
model,
voicy,378
jolie,53
tinyeco,22
desea,18
jolie&milk,8
jolieplus,8
idola,6


In [ ]:
# Set isComment Flag to True
youtube_comments_df['isComment'] = True

In [ ]:
# Set Source to youtube.com
youtube_comments_df['Source'] = 'youtube.com'

In [ ]:
# Add postCaption and postTimeStamp
api_service_name = "youtube"
api_version = "v3"
YOUTUBE_TOKEN = userdata.get('YOUTUBE_TOKEN')

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=YOUTUBE_TOKEN)

for video_url in youtube_comments_df.videoUrl.unique():
  id = video_url.split('=')[1].split('&')[0]

  request = youtube.videos().list(
      part="snippet,statistics",
      id=id
  )
  response = request.execute()

  description = response['items'][0].get('snippet', {}).get('description', pd.NA) #postCaption
  time = response['items'][0].get('snippet', {}).get('publishedAt', pd.NA) #postTimestamp


  # set the postCaption and the postTimestamp from response
  youtube_comments_df.loc[youtube_comments_df.videoUrl == video_url, 'postCaption'] = description
  youtube_comments_df.loc[youtube_comments_df.videoUrl == video_url, 'postTimeStamp'] = time

In [ ]:
# Rename videoUrl to postUrl
youtube_comments_df.rename(columns={'videoUrl': 'postUrl'}, inplace=True)

In [ ]:
youtube_comments_df.to_csv(data_folder+'youtube.csv', sep='\t', index=False)

## Postprocessing

In [ ]:
# Load comments data
digimind_df = pd.read_csv(data_folder + 'digimind.csv', sep='\t')
digimind_df = digimind_df[(digimind_df['Media type'] != 'Instagram') & (digimind_df['Media type'] != 'YouTube')]

instagram_df = pd.read_csv(data_folder + 'instagram.csv', sep='\t')

youtube_df = pd.read_csv(data_folder + 'youtube.csv', sep='\t')

In [ ]:
digimind_df.shape, instagram_df.shape, youtube_df.shape

((476, 9), (84, 9), (493, 9))

In [ ]:
sorted(instagram_df.columns) == sorted(youtube_df.columns), sorted(digimind_df.columns) == sorted(youtube_df.columns)

(True, True)

In [ ]:
# Concatenate all the records
social_media_comments_df = pd.concat([instagram_df, youtube_df, digimind_df], ignore_index=True)
social_media_comments_df.isna().sum(), social_media_comments_df.shape

(text                0
 postUrl             0
 commentTimeStamp    0
 postCaption         6
 postTimeStamp       0
 model               0
 Media type          0
 isComment           0
 Source              0
 dtype: int64,
 (1053, 9))

In [ ]:
# Translate text and postCaption columns
# Initialize the translator
translator = GoogleTranslator(target='en')

def is_full_emoji(text):
    """Check if the given text contains only emojis."""
    if not isinstance(text, str):
      return False
    text_without_emojis = emoji.replace_emoji(text, replace='')
    return len(text_without_emojis.strip()) == 0

def clean_text(text):
    """Clean the input text by removing extra spaces."""
    if pd.isna(text):
      return text
    # Remove leading/trailing spaces and replace newlines with single spaces
    text = text.strip().replace('\n', ' ')
    # Replace multiple spaces with single spaces
    text = re.sub(r'\s+', ' ', text)
    return text

def translate_text(text, retries=3):
    """Translate the given text to English, with retry logic in case of failure."""
    for attempt in range(retries):
      try:
        return translator.translate(text)
      except Exception:
        time.sleep(1)  # Wait before retrying
    return text  # Return original text if all retries fail

def preserve_emojis(text):
    """Replace emojis with placeholders and return the mapping dictionary."""
    emojis_found = re.findall(r':\w+:', text)
    emoji_dict = {emoji_name: f'__EMOJI{i}__' for i, emoji_name in enumerate(emojis_found)}
    for emoji_name, placeholder in emoji_dict.items():
      text = text.replace(emoji_name, placeholder)
    return text, emoji_dict

def restore_emojis(text, emoji_dict):
    """Restore emojis from placeholders using the provided dictionary."""
    for emoji_name, placeholder in emoji_dict.items():
      text = text.replace(placeholder, emoji_name)
    return text

def process_text(text):
    """Process text by handling emojis and translating it if necessary."""
    if pd.isna(text):
        return text

    if is_full_emoji(text):
        return emoji.demojize(text)  # Convert emojis to descriptive text

    text = emoji.demojize(text)  # Convert emojis to text format
    text, emoji_dict = preserve_emojis(text)  # Preserve emojis with placeholders
    text = translate_text(text)  # Translate the text
    return restore_emojis(text, emoji_dict)  # Restore emojis in the translated text

def clean_and_translate(row):
    """Clean and translate the comment and post caption based on context."""
    # Extract and clean the relevant fields
    comment = clean_text(row.iloc[0]) #comment
    post_caption = clean_text(row.iloc[3]) #postCaption
    is_comment = row.iloc[7] #isComment

    if not is_comment:  # If it's not a comment
        processed_comment = process_text(comment)
        row.iloc[0] = processed_comment
        row.iloc[3] = processed_comment  # Sync postCaption with comment
    else:  # If it's a comment
        row.iloc[0] = process_text(comment)
        row.iloc[3] = process_text(post_caption)

    return row

# Apply the function to the DataFrame
social_media_comments_df = social_media_comments_df.apply(clean_and_translate, axis=1)
social_media_comments_df.head()

,text,postUrl,commentTimeStamp,postCaption,postTimeStamp,model,Media type,isComment,Source
0,:red_heart:,https://www.instagram.com/p/Ct1EteaIVUd,2023-08-27T07:35:30.000Z,A tiny machine for mighty practicality! Order ...,23/06/2023 12:19:07,tinyeco,Instagram,True,instagram.com
1,I love the color,https://www.instagram.com/reel/CxtPqQLr5D9,2023-10-01T19:03:35.000Z,Coffee machine Here is finally our new coffee ...,27/09/2023 21:45:34,tinyeco,Instagram,True,instagram.com
2,:smiling_face_with_heart-eyes::raising_hands:,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-29T09:58:03.000Z,Coffee machine Here is finally our new coffee ...,27/09/2023 21:45:34,tinyeco,Instagram,True,instagram.com
3,I like the color. Different from usual,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-28T14:11:20.000Z,Coffee machine Here is finally our new coffee ...,27/09/2023 21:45:34,tinyeco,Instagram,True,instagram.com
4,Color :TOP_arrow:,https://www.instagram.com/reel/CxtPqQLr5D9,2023-09-28T12:09:52.000Z,Coffee machine Here is finally our new coffee ...,27/09/2023 21:45:34,tinyeco,Instagram,True,instagram.com


In [ ]:
social_media_comments_df.to_csv(data_folder+'comments.csv', sep='\t', index=False)

# Reviews

In [ ]:
# Load reviews
wonderflow_df1 = pd.read_excel(data_folder + 'Tiny Wonderflow.xlsx')
wonderflow_df2 = pd.read_excel(data_folder + 'Tiny Eco Wonderflow.xlsx')

wonderflow_df1.shape, wonderflow_df2.shape

((5072, 21), (145, 21))

In [ ]:
sorted(wonderflow_df1.columns) == sorted(wonderflow_df2.columns)

True

In [ ]:
wonderflow_df1.isna().sum()

,0
feedbackId,0
title,2215
originalTitle,3122
text,242
originalText,2733
name,0
brandName,0
nps,5072
price,5072
sku,0


In [ ]:
wonderflow_df2.isna().sum()

,0
feedbackId,0
title,66
originalTitle,140
text,15
originalText,136
name,0
brandName,0
nps,145
price,145
sku,0


In [ ]:
# Remove useless columns (where the number of nan values == shape[0])
wonderflow_df1 = wonderflow_df1.dropna(axis=1, how='all').reset_index(drop=True)
wonderflow_df2 = wonderflow_df2.dropna(axis=1, how='all').reset_index(drop=True)

In [ ]:
# Unique topics
def extract_unique_topics(df, pos_col, neg_col):
    # Drop rows with missing data in the topic columns
    pos_topic_df = df[pos_col].dropna().reset_index(drop=True)
    neg_topic_df = df[neg_col].dropna().reset_index(drop=True)

    # Extract unique positive and negative topics
    unique_pos_topics = set()
    unique_neg_topics = set()

    for topics in pos_topic_df:
        unique_pos_topics.update(topic.split('-')[0].strip() for topic in topics.split(','))
    for topics in neg_topic_df:
        unique_neg_topics.update(topic.split('-')[0].strip() for topic in topics.split(','))

    # Combine unique positive and negative topics
    return unique_pos_topics | unique_neg_topics


# Example usage with wonderflow_df1 and wonderflow_df2
unique_topics1 = extract_unique_topics(wonderflow_df1, "positiveTopics", "negativeTopics")
unique_topics2 = extract_unique_topics(wonderflow_df2, "positiveTopics", "negativeTopics")

# Combine topics from both dataframes
unique_topics = sorted(unique_topics1 | unique_topics2)

# Output the results
print(f"{len(unique_topics1)} unique topics from wonderflow_df1")
print(f"{len(unique_topics2)} unique topics from wonderflow_df2")
print(f"{len(unique_topics)} unique topics overall")

103 unique topics from wonderflow_df1
37 unique topics from wonderflow_df2
103 unique topics overall


In [ ]:
from pprint import pprint
pprint(unique_topics)

['Advertisement',
 'App',
 'App integration',
 'Availability',
 'Bitter',
 'Body',
 'Brand',
 'Brewing',
 'Bump during delivering',
 'Burnt',
 'Capsule',
 'Capsule case',
 'Capsules bio',
 'Capsules compostable',
 'Capsules container',
 'Capsules materials',
 'Capsules plastic',
 'Circuit breaker',
 'Coffee availability',
 'Cold',
 'Comparison',
 'Compatibility',
 'Connectivity',
 'Consistency',
 'Continuous fast',
 'Continuous slow',
 'Cord',
 'Crema',
 'Crema absent',
 'Customer service',
 'Descaling',
 'Design',
 'Difficult latch',
 'Difficult to pull out',
 'Display',
 'Dose',
 'Dose absent',
 'Dose less than standard',
 'Dose more than standard',
 'Drip tray',
 'Dripping',
 'Dripping after brewing',
 'Durability',
 'Failures',
 'Flow',
 'Go down',
 'Hot',
 'Intense',
 'Kit missing',
 'Leds',
 'Lever',
 'Logistics general',
 'Logistics mismatch',
 'Machine availability',
 'Machines materials',
 'Maintenance',
 'Mechanics',
 'Milk frother',
 'Noise',
 'Not extractable',
 'Not fit',


Alfter performing an external analysis we are going to cluster most of the 103 topics and we assign to them a specific class between:
- Coffee Quality
- Machine Performance
- Capsules and Compatibility
- Design and Materials
- User Experience
- Maintenance and Durability
- Services and Logistics
- Environmental Impact
- Brand Awareness
- Price & worth

In [ ]:
topic_classes = {
    "Coffee Quality": [
        "Bitter",
        "Body",
        "Burnt",
        "Crema",
        "Crema absent",
        "Dose",
        "Dose absent",
        "Dose less than standard",
        "Dose more than standard",
        "Hot",
        "Intense",
        "Performance coffee drink",
        "Powder in cup",
        "Smell coffee drink",
        "Taste",
        "Weak",
        "Weak/Watery"
    ],
    "Machine Performance": [
        "Brewing",
        "Cold",
        "Continuous fast",
        "Continuous slow",
        "Flow",
        "Hot",
        "Noise",
        "Overheating",
        "Spray",
        "Stopped",
        "Stuck at brewing end",
        "Temperature",
        "Switch-off issues",
        "Overall performance"
    ],
    "Capsules and Compatibility": [
        "Capsule",
        "Capsule case",
        "Capsules bio",
        "Capsules compostable",
        "Capsules container",
        "Capsules materials",
        "Capsules plastic",
        "Compatibility",
        "Coffee availability"
    ],
    "Design and Materials": [
        "Design",
        "Machines materials",
        "Size",
        "Weight",
        "Lever"
    ],
    "User Experience": [
        "App",
        "App integration",
        "Connectivity",
        "Customer service",
        "Display",
        "Smart system",
        "Usability",
        "User instruction",
        "User interface",
        "Overall satisfaction"
    ],
    "Maintenance and Durability": [
        "Circuit breaker",
        "Descaling",
        "Dripping",
        "Dripping after brewing",
        "Durability",
        "Maintenance",
        "Mechanics",
        "Milk frother",
        "Switch",
        "Vibrations",
        "Water leakage",
        "Water leakages during brewing",
        "Water tank",
        "Pump",
        "Failures",
        "Self-clean",
        "Drip tray",
        "Switch-off issues"
    ],
    "Services and Logistics": [
        "Advertisement",
        "Availability",
        "Bump during delivering",
        "Kit missing",
        "Logistics general",
        "Logistics mismatch",
        "Machine availability",
        "Package",
        "Retailer",
        "Voucher",
        "Wrong order preparation",
        "Coffee availability"
    ],
    "Environmental Impact": [
        "Capsules bio",
        "Capsules compostable",
        "Sustainability"
    ],
    "Brand Awareness": ["Brand"],
    "Price & worth": ["Price & worth"]
}

In [ ]:
# Set positiveTopicClass and negativeTopicClass wrt positiveTopics and negativeTopics

model = SentenceTransformer('bert-base-uncased')  # Choose an appropriate model

class_embeddings = {}
for topic_class, class_topics in topic_classes.items():
    class_embeddings[topic_class] = model.encode(' '.join(class_topics))

topic_embeddings = {}
for topic_class, class_topics in topic_classes.items():
    for class_topic in class_topics:
        topic_embeddings[class_topic] = model.encode(class_topic)


def assign_most_similar_class(topic):
    # Embed the input topic
    topic_embedding = model.encode(topic)

    # Calculate similarities
    max_similarity = -1
    most_similar_class = None

    for topic_class, class_embedding in class_embeddings.items():
        similarity = cosine_similarity(topic_embedding.reshape(1, -1), class_embedding.reshape(1, -1))[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_class = topic_class

    for class_topic, class_topic_embedding in topic_embeddings.items():
        similarity = cosine_similarity(topic_embedding.reshape(1, -1), class_topic_embedding.reshape(1, -1))[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            # Find the class of the most similar topic
            for topic_class, class_topics in topic_classes.items():
                if class_topic in class_topics:
                    most_similar_class = topic_class
                    break

    # Return the most similar class or 'Other' if below a threshold
    return most_similar_class

def assign_topic_class(topics_str, topic_classes):
    if pd.isnull(topics_str):  # Handle NaN values
        return np.NaN
    topics = [topic.split(' - ')[0].strip() for topic in topics_str.split(',')]
    topic_classes_list = []
    for topic in topics:
        for topic_class, class_topics in topic_classes.items():
            if topic in class_topics:
                topic_classes_list.append(topic_class)
                break  # Move to the next topic once a class is assigned
        else:  # If no class is found for the topic
            topic_classes_list.append(assign_most_similar_class(topic))
    topic_classes_list = list(set(topic_classes_list))
    return ', '.join(topic_classes_list)


wonderflow_df1['positiveTopicClass'] = wonderflow_df1['positiveTopics'].apply(lambda x: assign_topic_class(x, topic_classes))
wonderflow_df1['negativeTopicClass'] = wonderflow_df1['negativeTopics'].apply(lambda x: assign_topic_class(x, topic_classes))

wonderflow_df2['positiveTopicClass'] = wonderflow_df2['positiveTopics'].apply(lambda x: assign_topic_class(x, topic_classes))
wonderflow_df2['negativeTopicClass'] = wonderflow_df2['negativeTopics'].apply(lambda x: assign_topic_class(x, topic_classes))

In [ ]:
wonderflow_df1.head()

,feedbackId,title,originalTitle,text,originalText,name,brandName,sku,country,channel,feedbackRating,badges,publishedByUserAt,purchaseDate,images,positiveTopics,negativeTopics,positiveTopicClass,negativeTopicClass
0,5f2287a7023b9135c0bbacf2,The noise is horrendous,NaN,Not fit tor purpose sounds like Concorde takin...,NaN,Tiny,Lavazza,"Tiny White, Tiny Black",United Kingdom,amazon.co.uk,1.0,is verified purchase,2020-03-12,NaT,NaN,NaN,NaN,NaN,NaN
1,5f228af1023b9135c0bd9fa8,😍😍😍😍😍👍🏼,NaN,😍😍😍😍👍🏼,NaN,Tiny,Lavazza,Tiny White,United Kingdom,amazon.co.uk,5.0,is verified purchase,2019-02-08,NaT,NaN,NaN,NaN,NaN,NaN
2,5f228af1023b9135c0bd9fe3,5*,5*,5 *****,5*,Tiny,Lavazza,Tiny White,United Kingdom,amazon.co.uk,5.0,is verified purchase,2019-10-15,NaT,NaN,NaN,NaN,NaN,NaN
3,5f228b09023b9135c0bdab3a,Does the job well.,NaN,And for very little money.,NaN,Tiny,Lavazza,"Tiny White, Tiny Black",United Kingdom,amazon.co.uk,5.0,is verified purchase,2019-11-04,NaT,NaN,NaN,NaN,NaN,NaN
4,5f22e5a4023b9135c0ff5629,Mediocre.,Mittelmäßig,"Unfortunately, only small espresso cups fit un...",Leider passen nur die kleinen Espresso Tassen ...,Tiny,Lavazza,Tiny Black,Germany,amazon.de,3.0,is verified purchase,2019-05-04,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
wonderflow_df2.head()

,feedbackId,title,originalTitle,text,originalText,name,brandName,sku,country,channel,feedbackRating,badges,publishedByUserAt,purchaseDate,images,positiveTopics,negativeTopics,positiveTopicClass,negativeTopicClass
0,6409d870692b4cc13df2a70b,NaN,NaN,NaN,NaN,NEW TINY ECO,Lavazza,"Tiny Eco White, Tiny Eco Red, Tiny Eco Black",Italy,unieuro.it,5,NaN,2022-12-26,NaT,NaN,NaN,NaN,NaN,NaN
1,6409d870692b4cc13df2a783,NaN,NaN,NaN,NaN,NEW TINY ECO,Lavazza,"Tiny Eco White, Tiny Eco Red, Tiny Eco Black",Italy,unieuro.it,5,is verified purchase,2023-01-04,2022-12-06 23:00:00,NaN,NaN,NaN,NaN,NaN
2,658e85eaacfbcc5f0f98a1f4,NaN,NaN,NaN,NaN,NEW TINY ECO,Lavazza,"Tiny Eco Red, Tiny Eco Black, Tiny Eco White",Italy,unieuro.it,5,NaN,2023-08-19,NaT,NaN,NaN,NaN,NaN,NaN
3,658e85eaacfbcc5f0f98a1fa,NaN,NaN,NaN,NaN,NEW TINY ECO,Lavazza,"Tiny Eco Red, Tiny Eco White, Tiny Eco Black",Italy,unieuro.it,4,NaN,2024-08-29,NaT,NaN,NaN,NaN,NaN,NaN
4,658e85eaacfbcc5f0f98a216,NaN,NaN,NaN,NaN,NEW TINY ECO,Lavazza,"Tiny Eco Black, Tiny Eco White, Tiny Eco Red",Italy,unieuro.it,5,NaN,2023-08-11,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Keep these columns: title, text, name, channel, feedbackRating, publishedByUserAt
wonderflow_df1 = wonderflow_df1[['title', 'text', 'name', 'channel', 'feedbackRating', 'publishedByUserAt', 'positiveTopicClass', 'negativeTopicClass']]
wonderflow_df2 = wonderflow_df2[['title', 'text', 'name', 'channel', 'feedbackRating', 'publishedByUserAt', 'positiveTopicClass', 'negativeTopicClass']]

In [ ]:
# Concat title and text
wonderflow_df1.loc[:, 'text'] = wonderflow_df1['title'] + ' ' + wonderflow_df1['text']
wonderflow_df2.loc[:, 'text'] = wonderflow_df2['title'] + ' ' + wonderflow_df2['text']

In [ ]:
wonderflow_df1['name'].unique(), wonderflow_df2['name'].unique()

(array(['Tiny'], dtype=object), array(['NEW TINY ECO'], dtype=object))

In [ ]:
# Change name with model ('NEW TINY ECO->tinyeco, TINY->tiny)
wonderflow_df1['model'] = wonderflow_df1['name'].str.replace('Tiny', 'tiny')
wonderflow_df2['model'] = wonderflow_df2['name'].str.replace('NEW TINY ECO', 'tinyeco')

In [ ]:
# Change publishedByUserAt into reviewDate
wonderflow_df1.rename(columns={'publishedByUserAt': 'reviewDate'}, inplace=True)
wonderflow_df2.rename(columns={'publishedByUserAt': 'reviewDate'}, inplace=True)

In [ ]:
# Remove these columns: title, name
wonderflow_df1.drop(columns=['title', 'name'], inplace=True)
wonderflow_df2.drop(columns=['title', 'name'], inplace=True)

In [ ]:
# Concat the two dataframe
wonderflow_df = pd.concat([wonderflow_df1, wonderflow_df2], ignore_index=True)
wonderflow_df.isna().sum(), wonderflow_df.shape

(text                  2287
 channel                  0
 feedbackRating           0
 reviewDate               0
 positiveTopicClass     825
 negativeTopicClass    3726
 model                    0
 dtype: int64,
 (5217, 7))

In [ ]:
# Remove the rows where text in NaN
wonderflow_df.dropna(subset=['text'], inplace=True).reset_index(drop=True)
wonderflow_df.shape, wonderflow_df.isna().sum()

((2930, 7),
 text                     0
 channel                  0
 feedbackRating           0
 reviewDate               0
 positiveTopicClass     325
 negativeTopicClass    2224
 model                    0
 dtype: int64)

In [ ]:
# Translate reviews like comments
def clean_and_translate(x):
    return process_text(x)

wonderflow_df['text'] = wonderflow_df['text'].apply(clean_and_translate)
wonderflow_df.head()

,text,channel,feedbackRating,reviewDate,positiveTopicClass,negativeTopicClass,model
0,The noise is horrendous Not fit tor purpose so...,amazon.co.uk,1.0,2020-03-12,NaN,NaN,tiny
1,:smiling_face_with_heart-eyes::smiling_face_wi...,amazon.co.uk,5.0,2019-02-08,NaN,NaN,tiny
2,5* 5 *****,amazon.co.uk,5.0,2019-10-15,NaN,NaN,tiny
3,Does the job well. And for very little money.,amazon.co.uk,5.0,2019-11-04,NaN,NaN,tiny
4,"Mediocre. Unfortunately, only small espresso c...",amazon.de,3.0,2019-05-04,NaN,NaN,tiny


In [ ]:
wonderflow_df.to_csv(data_folder + 'reviews.csv', sep='\t', index=False)